In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from dotenv import load_dotenv
load_dotenv()                      # 1️⃣  make env-vars visible

from src.containers import Container   # 2️⃣  class body reads env once

import logging
logging.basicConfig(level=logging.INFO)

container = Container()            # 3️⃣  build the container

weaviate_helper      = container.weaviate_helper()
late_chunking_helper = container.late_chunking_helper()
ingest_helper = container.ingest_helper()


INFO:src.embeddings.late_chunking:Initialized LateChunkingHelper: device=cuda, max_chunk_chars=2048
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jinaai/jina-embeddings-v3
INFO:sentence_transformers.SentenceTransformer:5 prompts are loaded, with the keys: ['retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching']


In [24]:
ingest_helper._scan_and_get_directories("knowledge\\kubernetes\\website-main\\content\\en\\examples", {"include": ["yaml", "yml"], "exclude": [], "min_length": 0, "max_length": -1, "generate_summary": True})

INFO:src.ingest.ingest_helper:File 'knowledge\kubernetes\website-main\content\en\examples\admin\namespace-dev.json' skipped. Reason: File extension 'json' not in include list ['yaml', 'yml']
INFO:src.ingest.ingest_helper:File 'knowledge\kubernetes\website-main\content\en\examples\admin\namespace-prod.json' skipped. Reason: File extension 'json' not in include list ['yaml', 'yml']
INFO:src.ingest.ingest_helper:File 'knowledge\kubernetes\website-main\content\en\examples\admin\resource\memory-available-cgroupv2.sh' skipped. Reason: File extension 'sh' not in include list ['yaml', 'yml']
INFO:src.ingest.ingest_helper:File 'knowledge\kubernetes\website-main\content\en\examples\admin\resource\memory-available.sh' skipped. Reason: File extension 'sh' not in include list ['yaml', 'yml']
INFO:src.ingest.ingest_helper:File 'knowledge\kubernetes\website-main\content\en\examples\application\job\rabbitmq\Dockerfile' skipped. Reason: File extension 'Dockerfile' not in include list ['yaml', 'yml']
IN

***************************************
knowledge\kubernetes\website-main\content\en\examples\access
***************************************
***************************************
knowledge\kubernetes\website-main\content\en\examples\access\certificate-signing-request
***************************************
***************************************
knowledge\kubernetes\website-main\content\en\examples\access\certificate-signing-request\clusterrole-approve.yaml
***************************************
***************************************
knowledge\kubernetes\website-main\content\en\examples\access\certificate-signing-request\clusterrole-create.yaml
***************************************
***************************************
knowledge\kubernetes\website-main\content\en\examples\access\certificate-signing-request\clusterrole-sign.yaml
***************************************
***************************************
knowledge\kubernetes\website-main\content\en\examples\access\deployment-

([{'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-approve.yaml',
   'content': '',
   'file_extension': 'yaml'},
  {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-create.yaml',
   'content': '',
   'file_extension': 'yaml'},
  {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-sign.yaml',
   'content': '',
   'file_extension': 'yaml'},
  {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\deployment-replicas-policy.yaml',
   'content': '',
   'file_extension': 'yaml'},
  {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\endpoints-aggregated.yaml',
   'content': '',
   'file_extension': 'yaml'},
  {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\image-matches-namespace-environment.policy.yaml',
   '

In [30]:
with weaviate_helper.connect() as client:
    collection = client.collections.get(name="kubernetes_code")
    for obj in collection.iterator():
        print(obj)

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


Object(uuid=_WeaviateUUIDInt('0012f0f6-fb11-5f88-8507-3374b64724b5'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': '# This content describes a Kubernetes Pod configuration in YAML, focusing on setting up a liveness probe for container health monitoring. The Pod is named "liveness-exec" and contains a single container running the BusyBox image. The container executes a shell command that creates a file (`/tmp/healthy`), waits for 30 seconds, deletes the file, and then sleeps for an additional 600 seconds.\n\nThe key feature is the liveness probe, which periodically checks if the container is healthy by executing the `cat /tmp/healthy` command. If the file exists, the command succeeds, indicating the container is alive; if the file has been removed, the probe fails, prompting Kubernetes to restart the container. The probe starts after an init

In [25]:
ingest_helper.ingest_knowledge()

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_fundamentals "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/Kubernetes_fundamentals "HTTP/1.1 200 OK"
INFO:src.ingest.ingest_helper:Collection 'kubernetes_fundamentals' deleted.
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_code "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/Kubernetes_code "HTTP/1.1 200 OK"
INFO:src.ingest.ingest_helper:Collection 'kubernetes_code' deleted.
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.o

***************************************
[{'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\01-prerequisites.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\02-jumpbox.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\03-compute-resources.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\04-certificate-authority.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\05-kubernetes-configuration-files.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\06-data-encryption-keys.md', 'content': '', 'file_extension': 'md'}, {'path': 'knowledge\\kubernetes\\kubernetes-the-hard-way-master\\docs\\07-bootstrapping-etcd.md', 'content': '', 'file_extension': 

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.84it/s]
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: completed embeddings for 2 docs
INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge\kubernetes\kubernetes-the-hard-way-master\docs\02-jumpbox.md
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: docs=3
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.98it/s]
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: completed embeddings for 3 docs
INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge\kubernetes\kubernetes-the-hard-way-master\docs\03-compute-resources.md
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: docs=8
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.83it/s]
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: completed embeddings for 8 docs
INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge\kubernetes\kubernetes-the-hard-w

***************************************
[{'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-approve.yaml', 'content': '', 'file_extension': 'yaml'}, {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-create.yaml', 'content': '', 'file_extension': 'yaml'}, {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\certificate-signing-request\\clusterrole-sign.yaml', 'content': '', 'file_extension': 'yaml'}, {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\deployment-replicas-policy.yaml', 'content': '', 'file_extension': 'yaml'}, {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\endpoints-aggregated.yaml', 'content': '', 'file_extension': 'yaml'}, {'path': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\access\\image-matches-namespace-environment.policy.yaml', 'cont

RuntimeError: This event loop is already running

In [ ]:
with weaviate_helper.connect() as client:
    client.collections.delete(name="test")

weaviate_helper.create_collection(collection_name="test", description="test description")

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


In [ ]:
docs, canonical_doc = late_chunking_helper.chunk_file(
    path="knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md",
)
docs = late_chunking_helper.generate_late_chunking_embeddings(docs, canonical_doc)

INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: docs=23
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.41it/s]
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: completed embeddings for 23 docs


In [ ]:
weaviate_helper.batch_insert(docs, collection_name="test")

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


In [31]:
query = "deployment with horizontal scaling"
query_embedding = late_chunking_helper.generate_query_embedding(query)
rag_results = weaviate_helper.rag_query(collection_name="kubernetes_code", query_embedding=query_embedding, limit=100)
rag_results = [rag_result.properties for rag_result in rag_results]
len(rag_results)


INFO:src.embeddings.late_chunking:generate_query_embedding: query='deployment with horizontal scaling'
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_code "HTTP/1.1 200 OK"


100

In [32]:
ranked_results = late_chunking_helper.re_rank(query=query, docs=rag_results)
ranked_results

INFO:src.embeddings.late_chunking:re_rank: 100 docs, top_k=None
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.33it/s]
INFO:src.embeddings.late_chunking:re_rank: returning 100 ranked docs


[({'content': '# This YAML configuration defines a Horizontal Pod Autoscaler (HPA) for a Kubernetes deployment, specifically targeting a ReplicaSet named "frontend." The HPA automatically adjusts the number of pod replicas between 3 and 10, based on CPU utilization. It aims to maintain CPU usage at around 50%, scaling out or in to ensure the application remains responsive while efficiently using resources. This setup helps improve application scalability and reliability by dynamically managing pod counts according to workload demand.\napiVersion: autoscaling/v1\nkind: HorizontalPodAutoscaler\nmetadata:\n  name: frontend-scaler\nspec:\n  scaleTargetRef:\n    kind: ReplicaSet\n    name: frontend\n  minReplicas: 3\n  maxReplicas: 10\n  targetCPUUtilizationPercentage: 50\n',
   'filename': 'knowledge\\kubernetes\\website-main\\content\\en\\examples\\controllers\\hpa-rs.yaml',
   'summary': 'This YAML configuration defines a Horizontal Pod Autoscaler (HPA) for a Kubernetes deployment, speci

In [ ]:
from weaviate.classes.query import Filter

with weaviate_helper.connect() as client:
    collection = client.collections.get(name="test")
    response = collection.query.fetch_objects(
    filters=Filter.by_property("filename").like("*quick-reference.md*"),
    limit=3
)
response.objects

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
d:\Python\MasterIA\TFM\test_folder\.venv\lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


[Object(uuid=_WeaviateUUIDInt('5dd3f31c-5e73-4bdb-b099-e7fcf1eb1acb'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': '## {{% heading "whatsnext" %}}  \n* Read the [kubectl overview](/docs/reference/kubectl/) and learn about [JsonPath](/docs/reference/kubectl/jsonpath).  \n* See [kubectl](/docs/reference/kubectl/kubectl/) options.  \n* Also read [kubectl Usage Conventions](/docs/reference/kubectl/conventions/) to understand how to use kubectl in reusable scripts.  \n* See more community [kubectl cheatsheets](https://github.com/dennyzhang/cheatsheet-kubernetes-A4).', 'filename': 'knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md', 'chunk': '23/23', 'subchunk': '1/1'}, references=None, vector={}, collection='Test'),
 Object(uuid=_WeaviateUUIDInt('dc2a97aa-b075-49f4-a7ca-1efaeb886585'), metadata=MetadataRetur